# Implementing a Backgammon Player

#### Vinícius Miranda

Adapted from Aimacode. (2018). games.py [Python code]. GitHub Repository. Retrieved from https://github.com/aimacode/aima-python/blob/master/games.py

In [1]:
from collections import namedtuple
import itertools
import random
import copy
import time
import operator
import numpy as np
from cachetools import cached, LRUCache

BackgammonState = namedtuple('BackgammonState', 'to_move, utility, board, moves, chance')

In [3]:
class Backgammon:
    """A two player game where the goal of each player is to move all the
    checkers off the board. The moves for each state are determined by
    rolling a pair of dice."""

    def __init__(self):
        """Initial state of the game"""
        
        point = {'W' : 0, 'B' : 0}
        board = [point.copy() for index in range(24)]
        board[0]['B'] = board[23]['W'] = 2
        board[5]['W'] = board[18]['B'] = 5
        board[7]['W'] = board[16]['B'] = 3
        board[11]['B'] = board[12]['W'] = 5
        
        # The last point (index 24) will be the bar
        # and stores captured pieces
        board.append(point.copy())        
        
        # All possible dice rolls
        self.ROLLS = list(itertools.combinations_with_replacement([1, 2, 3, 4, 5, 6], 2))
        
        # Bear off is the last phase of the game
        # where players can remove their pieces
        # from the board
        self.allow_bear_off = {'W' : False, 'B' : False}
        self.direction = {'W' : -1, 'B' : 1}
        
        self.initial = BackgammonState(to_move="W",
                                           utility=0,
                                           board=board,
                                           moves=self.get_all_moves(board, 'W'), chance=None)
    
    def actions(self, state):
        """Return a list of legal moves for a state."""
        player = state.to_move
        moves = state.moves
        
        if len(moves) == 1 and len(moves[0]) == 1:
            return moves
        legal_moves = []
        for move in moves:
            board = copy.deepcopy(state.board)
            legal_move = self.is_legal_move(board, move, state.chance, player)
            if legal_move:
                legal_moves.append(legal_move)

        return legal_moves

    def result(self, state, move):
        """Returns the resulting state after move is executed."""
        
        board = copy.deepcopy(state.board)
        player = state.to_move
        if move is not None:
            self.move_checker(board, move[0], state.chance[0], player)
            if len(move) == 2:
                self.move_checker(board, move[1], state.chance[1], player)
        to_move = ('W' if player == 'B' else 'B')
        return BackgammonState(to_move=to_move,
                                   utility=self.compute_utility(board, move, player),
                                   board=board,
                                   moves=self.get_all_moves(board, to_move), chance=None)

    def utility(self, state, player):
        """Return the value to player; 1 for win, -1 for loss, 0 otherwise."""
        return state.utility if player == 'W' else -state.utility

    def terminal_test(self, state):
        """A state is terminal if one player wins."""
        return state.utility != 0

    def get_all_moves(self, board, player):
        """All possible moves for a player i.e. all possible ways of
        choosing two checkers of a player from the board for a move
        at a given state."""
        all_points = board
        taken_points = [index for index, point in enumerate(all_points)
                        if point[player] > 0]
        
        if len(taken_points) == 1 and self.checkers_at_home(board, player) == 1:
            return [(taken_points[0], )]
        
        # The movements are given by all possible permutations of piece positions
        moves = list(itertools.permutations(taken_points, 2))
        moves = moves + [(index, index) for index, point in enumerate(all_points)
                         if point[player] >= 2]
        return moves

    def display(self, state):
        """Display state of the game."""
        board = state.board
        player = state.to_move
        print("bar   : \tW : ", board[24]['W'], "    B : ", board[24]['B'], "\n")
        for index, point in zip(range(24), board[:24]):
            print("point : ", index, "\tW : ", point['W'], "    B : ", point['B'])
                
        print("to play : ", player)

    def compute_utility(self, board, move, player):
        """If 'W' wins with this move, return 1; if 'B' wins return -1; else return 0."""
        util = {'W' : 1, 'B' : -1}
        for idx in range(0, 24):
            if board[idx][player] > 0:
                return 0
        return util[player]

    def checkers_at_home(self, board, player):
        """Return the no. of checkers at home for a player.
           White's home is at 0-5, whereas Black's at 18-24."""

        sum_range = range(6) if player == 'W' else range(18, 24)
        count = 0
        for idx in sum_range:
            count = count + board[idx][player]
        return count

    def is_legal_move(self, board, start, steps, player):
        """Move is a tuple which contains starting points of checkers to be
        moved during a player's turn. An on-board move is legal if both the destinations
        are open. A bear-off move is the one where a checker is moved off-board.
        It is legal only after a player has moved all his checkers to his home."""
        
        # We need a removal flag in the case that we are trying to
        # reintroduce a piece into the game
        removal_flag = False
        
        # If there are any checkers at the bar, they must be removed.
        if board[24][player] > 0:
            removal_flag = True
            if 24 not in start:
                return False
            
            # For player "B", the bar starting positiong is -1
            start_pos = 24 if player == "W" else -1
            
            if board[24][player] == 1:
                if start[0] == 24:
                    start = (start_pos, start[1])
                else:
                    start = (start[0], start_pos)
            if board[24][player] > 1:
                start = (start_pos, start_pos)
        
        dest1, dest2 = vector_add(start, steps)
        dest_range = range(0, 24)
        move1_legal = move2_legal = False            
        
        # The first movement is executed to consider the second move appropriately
        if dest1 in dest_range:
            if self.is_point_open(player, board[dest1]):
                self.move_checker(board, start[0], steps[0], player)
                move1_legal = True
        else:
            if self.allow_bear_off[player]:
                self.move_checker(board, start[0], steps[0], player)
                move1_legal = True
        if not move1_legal:
            return False
        
        if dest2 in dest_range:
            if self.is_point_open(player, board[dest2]):
                move2_legal = True
        else:
            if self.allow_bear_off[player]:
                move2_legal = True
                
        # If we are trying to remove a piece from the bar,
        # it's important to allow partial movements.
        if move1_legal and not move2_legal and removal_flag:
            return (start[0], )
            
        if move1_legal and move2_legal:
            return start

    def move_checker(self, board, start, steps, player):
        """Move a checker from starting point by a given number of steps"""    
        
        board[start][player] -= 1
        
        dest = start + steps
        dest_range = range(0, 24)
        
        if dest in dest_range:            
            board[dest][player] += 1
            opponent = 'B' if player == 'W' else 'W'
            
            # Since the movement is legal, the opponent has either 0 or 1 pieces
            if board[dest][opponent] == 1:  
                board[dest][opponent] = 0  # The piece is captured
                board[24][opponent]  += 1  # and is added to the bar
            
        if self.checkers_at_home(board, player) == 15:
            self.allow_bear_off[player] = True

    def is_point_open(self, player, point):
        """A point is open for a player if the no. of opponent's
        checkers already present on it is 0 or 1. A player can
        move a checker to a point only if it is open."""
        opponent = 'B' if player == 'W' else 'W'
        return point[opponent] <= 1

    def chances(self, state):
        """Return a list of all possible dice rolls at a state."""
        return self.ROLLS

    def outcome(self, state, chance):
        """Return the state which is the outcome of a dice roll."""
        dice = tuple(map((self.direction[state.to_move]).__mul__, chance))
        return BackgammonState(to_move=state.to_move,
                                   utility=state.utility,
                                   board=state.board,
                                   moves=state.moves, chance=dice)

    def probability(self, chance):
        """Return the probability of occurence of a dice roll."""
        return 1/36 if chance[0] == chance[1] else 1/18
    
    def to_move(self, state):
        """Return the player whose move it is in this state."""
        return state.to_move
    
    def play_game(self, player1, player2, prnt=True):
        state = self.initial
        self.turn = 1

        while True:
            for player in (player1, player2):
                if prnt: print("Turn: {}".format(self.turn))
                
                chance = random.choice(self.chances(state)) 
                state = self.outcome(state, chance)
                move = player(self, state)
                state = self.result(state, move)
                if self.terminal_test(state):
                    if prnt: self.display(state)
                    return self.utility(state, self.to_move(self.initial))
                self.turn += 1

In [4]:
cache = LRUCache(maxsize=128)
# The cache should be keyed by the game state, represented now as a tuple of two-element tuples
@cached(cache, key=lambda game, state: tuple(tuple(point.values()) for point in bg.initial.board))
def bg_eval(game, state):
    '''A simple cached evaluation function. It takes into account the position of pieces,
    vulnerable (single) pieces, how many checkers have been removed from the game, how
    many pieces are at the home board, and captures. Captures weigh heavily on the evalution.'''

    if game.terminal_test(state):
        return 1000*game.utility(state, game.to_move(state))

    maxplayer, minplayer = "W", "B"
    board = state.board
    max_in_game = min_in_game = 0
    max_singles = min_singles = 0
    max_points = min_points = 0
    max_bar = min_bar = 0
    
    # Number of pieces out of the game
    for idx in range(0, 24):
        max_in_game += board[idx][maxplayer]
        min_in_game += board[idx][minplayer]
        
        # Number of points with single pieces
        if board[idx][maxplayer] == 1:
            max_singles += 1
        if board[idx][minplayer] == 1:
            min_singles += 1
    
        # Number of points
        max_points += (24 - idx)/5*board[idx][maxplayer]
        min_points -= (1  + idx)/5*board[idx][minplayer]
    
    max_out_game = 15 - max_in_game
    min_out_game = 15 - min_in_game
    
    # Number of pieces at home board
    max_home = game.checkers_at_home(state.board, "W")
    min_home = game.checkers_at_home(state.board, "B")  
    
    # Number of pieces captured
    max_bar = board[24][maxplayer]
    min_bar = board[24][minplayer]
    
    max_overall = max_points + max_out_game * 5 + max_home * 2 - max_singles * 2 - max_bar * 50
    min_overall = min_points - min_out_game * 5 - min_home * 2 + min_singles * 2 + min_bar * 50
    
    return(max_overall  + min_overall)

In [5]:
def query_player(game, state):
    """Make a move by querying standard input."""
    print("\ncurrent state:")
    game.display(state)
    die1, die2 = state.chance
    print("Dice results: {} and {}.".format(abs(die1), abs(die2)))
    print("available moves: {}".format(state.moves))
    print("")
    move = None
    if game.actions(state):
        move_string = input('Your move? ')
        try:
            move = eval(move_string)
        except NameError:
            move = move_string
    else:
        print('no legal moves: passing turn to next player')
    return move


def random_player(game, state):
    """A player that chooses a legal move at random."""
    move = random.choice(game.actions(state)) if game.actions(state) else None
    return move

def expectiminimax_player(game, state):
    return expectiminimax(state, game, d = 4, eval_fn = bg_eval)

def pruned_expectiminimax_player(game, state):
    return expectiminimax_w_pruning(state, game, d = 4, eval_fn = bg_eval)

def vector_add(a, b):
    """Component-wise addition of two vectors."""
    return tuple(map(operator.add, a, b))

In [6]:
def expectiminimax(state, game, d=4, eval_fn=None):
    """Return the best move for a player after dice are thrown. The game tree
    includes chance nodes along with min and max nodes."""
    player = game.to_move(state)
    cutoff_test = lambda state, depth: depth > d or game.terminal_test(state)

    def max_value(state, depth):
        if cutoff_test(state, depth):
            return eval_fn(game, state)
        v = -np.inf
        for a in game.actions(state):
            v = max(v, chance_node(state, a, depth + 1))
        return v

    def min_value(state, depth):
        if cutoff_test(state, depth):
            return eval_fn(game, state)
        v = np.inf
        for a in game.actions(state):
            v = min(v, chance_node(state, a, depth + 1))
        return v
    
    fun = max_value if player == "W" else min_value
    fun_opponent = min_value if player == "W" else max_value

    def chance_node(state, action, depth):
        res_state = game.result(state, action)
        # Is the player the same as the root node's?
        chance_fun = fun if res_state.to_move == player else fun_opponent
        
        if cutoff_test(res_state, depth):
            return eval_fn(game, res_state)
        
        sum_chances = 0
        for chance in game.chances(res_state):
            res_state = game.outcome(res_state, chance)
            sum_chances += chance_fun(res_state, depth + 1) * game.probability(chance)
        return sum_chances 
    
    # Body of expectiminimax:
    root_fun = max if player == "W" else min
    return root_fun(game.actions(state),
                  key=lambda a: chance_node(state, a, 1), default=None)

def expectiminimax_w_pruning(state, game, d=4, eval_fn=None):
    """Similar to expectiminimax, but prunes max and min nodes."""
    player = game.to_move(state)
    cutoff_test = lambda state, depth: depth > d or game.terminal_test(state)

    def max_value(state, alpha, beta, depth):
        if cutoff_test(state, depth):
            return eval_fn(game, state)
        v = -np.inf
        for a in game.actions(state):
            v = max(v, chance_node(state, a, alpha, beta, depth + 1))
            if v >= beta:
                return v
            alpha = max(alpha, v)
        return v

    def min_value(state, alpha, beta, depth):
        if cutoff_test(state, depth):
            return eval_fn(game, state)
        v = np.inf
        for a in game.actions(state):
            v = min(v, chance_node(state, a, alpha, beta, depth + 1))
            if v <= alpha:
                return v
            beta = min(beta, v)
        return v
    
    fun = max_value if player == "W" else min_value
    fun_opponent = min_value if player == "W" else max_value

    def chance_node(state, action, alpha, beta, depth):
        res_state = game.result(state, action)
        # Is the player the same as the root node's?
        chance_fun = fun if res_state.to_move == player else fun_opponent
    
        if cutoff_test(res_state, depth):
            return eval_fn(game, res_state)
        
        sum_chances = 0
        for chance in game.chances(res_state):
            res_state = game.outcome(res_state, chance)
            sum_chances += chance_fun(res_state, alpha, beta, depth + 1) * game.probability(chance)

        return sum_chances 
    
    # Body of search
    alpha = -np.inf
    beta = np.inf
    best_score = -np.inf if player == "W" else np.inf
    best_action = None
    
    for a in game.actions(state):  
        
        if player == "W":  # If it's max
            v = chance_node(state, a, best_score, beta, 1)
            if v > best_score:
                best_score = v
                best_action = a
        
        else:  # If it's min
            v = chance_node(state, a, alpha, best_score, 1)
            if v < best_score:
                best_score = v
                best_action = a
    
    return best_action


# Depth vs. Run time Analysis

Testing the run time for different depths.

In [7]:
minimax_times = []
pruned_times = []
AI1VSrandom = []
AI3VSrandom = []

bg = Backgammon()
for i in range(1, 1):
    
    def expectiminimax_player(game, state):
        return expectiminimax(state, game, d = i, eval_fn = bg_eval)
    
    def pruned_expectiminimax_player(game, state):
        return expectiminimax_w_pruning(state, game, d = i, eval_fn = bg_eval)
    
    minimax_times_temp = []
    pruned_times_temp = []
    for j in range(5):
        start = time.time()
        bg.play_game(expectiminimax_player, random_player, False)
        minimax_times_temp.append(time.time() - start)
        
        start = time.time()
        outcome = bg.play_game(pruned_expectiminimax_player, random_player, False)
        pruned_times_temp.append(time.time() - start)
        if i == 1:
            AI1VSrandom.append(outcome)
        elif i == 3:
            AI3VSrandom.append(outcome)    
        
    minimax_times.append((i, minimax_times_temp))
    pruned_times.append((i, pruned_times_temp))


In [10]:
# Measure only once for depth 4
start = time.time()

def pruned_expectiminimax_player(game, state):
    return expectiminimax_w_pruning(state, game, d = 4, eval_fn = bg_eval)
bg.play_game(pruned_expectiminimax_player, random_player, False)

pruned4time = time.time() - start

In [9]:
for i in range(1, 1):
    print("\nDepth: {}".format(i))
    print("\tAverage: {:.4f} s".format(np.mean(minimax_times[i-1])))
    print("\tPruned average: {:.4f} s".format(np.mean(pruned_times[i-1])))
    print("\tTimes: ", minimax_times[i-1])
    print("\tPruned Times: ", pruned_times[i-1])
  
#print("\nDepth: {}".format(4))
#print("\tSingle run pruned time: {:.4f} s".format(pruned4time))

# Player vs. Player Analysis

Testing the effectiveness of different players.

In [10]:
def AI1(game, state):
    return expectiminimax_w_pruning(state, game, d = 1, eval_fn = bg_eval)

def AI3(game, state):
    return expectiminimax_w_pruning(state, game, d = 3, eval_fn = bg_eval)

def AI4wPruning(game, state):
    return expectiminimax_w_pruning(state, game, d = 4, eval_fn = bg_eval)

In [11]:
randomVSrandom = []
AI1VSAI3 = []

for pl1, pl2, stor in zip([random_player, AI1],
                          [random_player, AI3], 
                          [randomVSrandom, AI1VSAI3]):
    for i in range(0):
        bg = Backgammon()
        outcome = bg.play_game(pl1, pl2, False)
        stor.append(outcome)


In [13]:
bg = Backgammon()
# bg.play_game(query_player, AI4wPruning)